# 🤖 Machine Learning - Predicciones Comercios Buenos Aires

## MIT LIFT Lab × UBA - Equipo GreenThunder

---

### 📋 Modelos Implementados:

1. **📈 Predicción de Crecimiento Comercial** - Clasificación binaria
2. **🏪 Score de Viabilidad/Supervivencia** - Clustering + Scoring
3. **💰 Predicción de Salario Ofrecido** - Regresión
4. **🌪️ Impacto de Factores Externos en Ventas** - Clasificación multiclase

---

### ⚠️ DISCLAIMER

**Los modelos presentados constituyen análisis estadísticos predictivos basados en patrones históricos. Estas proyecciones NO garantizan comportamiento futuro y deben interpretarse como estimaciones probabilísticas sujetas a variabilidad contextual y factores externos no capturados.**

## 📦 1. Setup - Instalación y Imports

In [ ]:
# Instalar librerías (solo en Colab)
# !pip install pandas numpy scikit-learn matplotlib seaborn imbalanced-learn

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import warnings
warnings.filterwarnings('ignore')

# Machine Learning
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier, GradientBoostingRegressor, RandomForestRegressor
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.cluster import KMeans
from sklearn.metrics import (
    classification_report, confusion_matrix, roc_auc_score, roc_curve,
    mean_squared_error, r2_score, mean_absolute_error,
    accuracy_score, precision_score, recall_score, f1_score
)
from sklearn.decomposition import PCA

# Configuración de gráficos
plt.style.use('dark_background')
sns.set_palette("husl")
%matplotlib inline

print("✅ Librerías importadas correctamente")

## 📂 2. Carga de Datos

**Instrucciones para Colab:**
1. Sube el archivo `datos_comercios.csv` usando el panel de archivos (📁)
2. O móntalo desde Google Drive

In [ ]:
# Opción 1: Archivo local
df = pd.read_csv('datos_comercios.csv')

# Opción 2: Desde Google Drive (descomenta si usas Drive)
# from google.colab import drive
# drive.mount('/content/drive')
# df = pd.read_csv('/content/drive/MyDrive/datos_comercios.csv')

print(f"📊 Dataset cargado: {df.shape[0]} filas × {df.shape[1]} columnas")
df.head()

In [ ]:
# Información del dataset
print("\n📋 Información general:")
df.info()

print("\n📊 Estadísticas descriptivas:")
df.describe()

## 🧹 3. Preprocesamiento y Feature Engineering

In [ ]:
# Crear copia para trabajar
data = df.copy()

# --- FEATURES DERIVADAS ---

# 1. Antigüedad del comercio
data['antiguedad'] = 2024 - data['año_apertura']
data['antiguedad'] = data['antiguedad'].clip(lower=0, upper=100)  # Limitar valores extremos

# 2. Horas de operación diaria
def calcular_horas(row):
    try:
        apertura = int(row['hs_apertura'])
        cierre = int(row['hs_cierre'])
        horas = cierre - apertura
        if horas < 0:
            horas += 24
        return horas
    except:
        return np.nan

data['horas_operacion'] = data.apply(calcular_horas, axis=1)

# 3. Tiene acceso a crédito (binario)
credito_cols = ['credits_bancos', 'credits_proveedor', 'credits_familia', 'credits_gobierno', 'credits_privado']
data['tiene_credito'] = (data[credito_cols].sum(axis=1) > 0).astype(int)

# 4. Nivel tecnológico (ordinal)
def nivel_tech(texto):
    if pd.isna(texto):
        return 0
    texto = str(texto).lower()
    if 'alto' in texto or 'avanzado' in texto:
        return 3
    elif 'moderado' in texto:
        return 2
    elif 'básico' in texto or 'basico' in texto:
        return 1
    else:
        return 0

data['nivel_tecnologia'] = data['tecnologia'].apply(nivel_tech)

# 5. Expectativas de ventas (ordinal)
def expectativas_ventas(texto):
    if pd.isna(texto):
        return 0
    texto = str(texto).lower()
    if 'mayores' in texto or 'mejor' in texto:
        return 1
    elif 'menores' in texto or 'peor' in texto:
        return -1
    else:
        return 0

data['expectativas_ventas_num'] = data['exp_ventas_3mes'].apply(expectativas_ventas)

# 6. Ventas vs mes anterior (ordinal)
def ventas_tendencia(texto):
    if pd.isna(texto):
        return 0
    texto = str(texto).lower()
    if 'mejor' in texto:
        return 1
    elif 'peor' in texto:
        return -1
    else:
        return 0

data['ventas_tendencia'] = data['venta_vs_mesantes'].apply(ventas_tendencia)

# 7. Local propio (binario)
data['local_propio'] = (data['local'].str.lower() == 'propio').astype(int)

# 8. Afectaciones (convertir a numérico)
afect_map = {'Nada': 0, 'Poco': 1, 'Algo': 2, 'Mucho': 3}
for col in ['afect_crimen', 'afect_credito', 'afect_precios', 'afect_compe']:
    data[f'{col}_num'] = data[col].map(afect_map).fillna(0)

# 9. Limpieza de salarios
def limpiar_salario(val):
    try:
        if pd.isna(val):
            return np.nan
        # Remover caracteres no numéricos
        cleaned = str(val).replace('$', '').replace('.', '').replace(',', '').replace(' ', '').strip()
        num = float(cleaned)
        # Filtrar rango razonable
        if 100000 <= num <= 15000000:
            return num
        return np.nan
    except:
        return np.nan

data['min_salario_clean'] = data['min_salario'].apply(limpiar_salario)

print("✅ Features creadas:")
print("   - antiguedad")
print("   - horas_operacion")
print("   - tiene_credito")
print("   - nivel_tecnologia (0-3)")
print("   - expectativas_ventas_num (-1, 0, 1)")
print("   - ventas_tendencia (-1, 0, 1)")
print("   - local_propio (0/1)")
print("   - afectaciones numéricas (0-3)")
print("   - min_salario_clean")
print("\n📊 Shape después de feature engineering:", data.shape)

---
# 📈 MODELO 1: Predicción de Crecimiento Comercial

**Objetivo**: Predecir si un comercio quiere expandirse/crecer

**Variable target**: `quiere_crezca` (0 = No, 1 = Sí)

**Algoritmo**: Random Forest Classifier

In [ ]:
# --- PREPARAR DATOS ---
print("🎯 MODELO 1: Predicción de Crecimiento\n")

# Features para el modelo
features_m1 = [
    'antiguedad',
    'cantidad_trabajadores',
    'ventas_tendencia',
    'expectativas_ventas_num',
    'tiene_credito',
    'nivel_tecnologia',
    'local_propio',
    'horas_operacion'
]

# Preparar dataset (solo filas con target válido)
df_m1 = data[features_m1 + ['quiere_crezca']].copy()
df_m1 = df_m1.dropna()
df_m1 = df_m1[df_m1['quiere_crezca'].isin([0, 1, 0.0, 1.0])]

X_m1 = df_m1[features_m1]
y_m1 = df_m1['quiere_crezca'].astype(int)

print(f"📊 Tamaño del dataset: {len(df_m1)} comercios")
print(f"✅ Quieren crecer: {y_m1.sum()} ({y_m1.mean()*100:.1f}%)")
print(f"❌ No quieren crecer: {len(y_m1) - y_m1.sum()} ({(1-y_m1.mean())*100:.1f}%)")

In [ ]:
# --- TRAIN/TEST SPLIT ---
X_train_m1, X_test_m1, y_train_m1, y_test_m1 = train_test_split(
    X_m1, y_m1, test_size=0.25, random_state=42, stratify=y_m1
)

# Escalar features
scaler_m1 = StandardScaler()
X_train_m1_scaled = scaler_m1.fit_transform(X_train_m1)
X_test_m1_scaled = scaler_m1.transform(X_test_m1)

print(f"🔀 Train set: {len(X_train_m1)} | Test set: {len(X_test_m1)}")

In [ ]:
# --- ENTRENAR MODELO ---
print("\n🤖 Entrenando Random Forest Classifier...\n")

rf_m1 = RandomForestClassifier(
    n_estimators=200,
    max_depth=10,
    min_samples_split=10,
    min_samples_leaf=5,
    random_state=42,
    n_jobs=-1
)

rf_m1.fit(X_train_m1, y_train_m1)

# Predicciones
y_pred_m1 = rf_m1.predict(X_test_m1)
y_pred_proba_m1 = rf_m1.predict_proba(X_test_m1)[:, 1]

print("✅ Modelo entrenado!")

In [ ]:
# --- MÉTRICAS ---
accuracy_m1 = accuracy_score(y_test_m1, y_pred_m1)
precision_m1 = precision_score(y_test_m1, y_pred_m1)
recall_m1 = recall_score(y_test_m1, y_pred_m1)
f1_m1 = f1_score(y_test_m1, y_pred_m1)
auc_m1 = roc_auc_score(y_test_m1, y_pred_proba_m1)

print("\n📊 MÉTRICAS DEL MODELO 1:")
print("=" * 50)
print(f"⚡ Accuracy:  {accuracy_m1:.3f}")
print(f"🎯 Precision: {precision_m1:.3f}")
print(f"📈 Recall:    {recall_m1:.3f}")
print(f"🔥 F1-Score:  {f1_m1:.3f}")
print(f"📊 AUC-ROC:   {auc_m1:.3f}")
print("=" * 50)

# Classification Report
print("\n📋 Classification Report:")
print(classification_report(y_test_m1, y_pred_m1, target_names=['No crece', 'Sí crece']))

In [ ]:
# --- VISUALIZACIONES ---
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('MODELO 1: Predicción de Crecimiento Comercial', fontsize=18, fontweight='bold', color='#4FC3F7')

# 1. Confusion Matrix
cm_m1 = confusion_matrix(y_test_m1, y_pred_m1)
sns.heatmap(cm_m1, annot=True, fmt='d', cmap='Blues', ax=axes[0,0], 
            xticklabels=['No crece', 'Sí crece'], 
            yticklabels=['No crece', 'Sí crece'])
axes[0,0].set_title('Matriz de Confusión', fontsize=14, color='#4FC3F7')
axes[0,0].set_ylabel('Real')
axes[0,0].set_xlabel('Predicho')

# 2. ROC Curve
fpr, tpr, _ = roc_curve(y_test_m1, y_pred_proba_m1)
axes[0,1].plot(fpr, tpr, color='#4FC3F7', linewidth=3, label=f'AUC = {auc_m1:.3f}')
axes[0,1].plot([0, 1], [0, 1], 'k--', linewidth=1)
axes[0,1].set_xlabel('False Positive Rate')
axes[0,1].set_ylabel('True Positive Rate')
axes[0,1].set_title('ROC Curve', fontsize=14, color='#4FC3F7')
axes[0,1].legend()
axes[0,1].grid(alpha=0.3)

# 3. Feature Importance
importances_m1 = pd.DataFrame({
    'feature': features_m1,
    'importance': rf_m1.feature_importances_
}).sort_values('importance', ascending=True)

axes[1,0].barh(importances_m1['feature'], importances_m1['importance'], color='#4FC3F7')
axes[1,0].set_xlabel('Importancia')
axes[1,0].set_title('Feature Importance', fontsize=14, color='#4FC3F7')
axes[1,0].grid(axis='x', alpha=0.3)

# 4. Distribución de probabilidades
axes[1,1].hist(y_pred_proba_m1[y_test_m1 == 0], bins=30, alpha=0.6, label='No crece', color='#FF6B6B')
axes[1,1].hist(y_pred_proba_m1[y_test_m1 == 1], bins=30, alpha=0.6, label='Sí crece', color='#00E676')
axes[1,1].set_xlabel('Probabilidad predicha')
axes[1,1].set_ylabel('Frecuencia')
axes[1,1].set_title('Distribución de Probabilidades', fontsize=14, color='#4FC3F7')
axes[1,1].legend()
axes[1,1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# --- GUARDAR RESULTADOS MODELO 1 ---
resultados_m1 = {
    'modelo': 'Predicción de Crecimiento Comercial',
    'tipo': 'clasificacion_binaria',
    'algoritmo': 'Random Forest Classifier',
    'metricas': {
        'accuracy': float(accuracy_m1),
        'precision': float(precision_m1),
        'recall': float(recall_m1),
        'f1_score': float(f1_m1),
        'auc_roc': float(auc_m1)
    },
    'feature_importance': importances_m1.sort_values('importance', ascending=False).to_dict('records'),
    'confusion_matrix': cm_m1.tolist(),
    'dataset_size': {
        'total': len(df_m1),
        'train': len(X_train_m1),
        'test': len(X_test_m1)
    }
}

print("✅ Resultados guardados en variable 'resultados_m1'")

---
# 💰 MODELO 2: Predicción de Salario Ofrecido

**Objetivo**: Predecir el salario mínimo que un comercio está dispuesto a pagar

**Variable target**: `min_salario_clean` (continua, en ARS)

**Algoritmo**: Gradient Boosting Regressor

In [ ]:
# --- PREPARAR DATOS ---
print("💰 MODELO 2: Predicción de Salario\n")

features_m2 = [
    'antiguedad',
    'cantidad_trabajadores',
    'ventas_tendencia',
    'nivel_tecnologia',
    'local_propio',
    'horas_operacion',
    'tiene_credito'
]

# Preparar dataset
df_m2 = data[features_m2 + ['min_salario_clean', 'tipo_comercio']].copy()
df_m2 = df_m2.dropna(subset=['min_salario_clean'])
df_m2 = df_m2.dropna(subset=features_m2)

# One-hot encoding para tipo_comercio
tipo_dummies = pd.get_dummies(df_m2['tipo_comercio'], prefix='tipo')
X_m2 = pd.concat([df_m2[features_m2], tipo_dummies], axis=1)
y_m2 = df_m2['min_salario_clean']

print(f"📊 Tamaño del dataset: {len(df_m2)} comercios")
print(f"💵 Salario promedio: ${y_m2.mean():,.0f} ARS")
print(f"💵 Salario mediana: ${y_m2.median():,.0f} ARS")
print(f"💵 Rango: ${y_m2.min():,.0f} - ${y_m2.max():,.0f} ARS")

In [ ]:
# --- TRAIN/TEST SPLIT ---
X_train_m2, X_test_m2, y_train_m2, y_test_m2 = train_test_split(
    X_m2, y_m2, test_size=0.25, random_state=42
)

print(f"🔀 Train set: {len(X_train_m2)} | Test set: {len(X_test_m2)}")

In [ ]:
# --- ENTRENAR MODELO ---
print("\n🤖 Entrenando Gradient Boosting Regressor...\n")

gbr_m2 = GradientBoostingRegressor(
    n_estimators=200,
    max_depth=5,
    learning_rate=0.1,
    random_state=42
)

gbr_m2.fit(X_train_m2, y_train_m2)

# Predicciones
y_pred_m2 = gbr_m2.predict(X_test_m2)

print("✅ Modelo entrenado!")

In [ ]:
# --- MÉTRICAS ---
r2_m2 = r2_score(y_test_m2, y_pred_m2)
rmse_m2 = np.sqrt(mean_squared_error(y_test_m2, y_pred_m2))
mae_m2 = mean_absolute_error(y_test_m2, y_pred_m2)
mape_m2 = np.mean(np.abs((y_test_m2 - y_pred_m2) / y_test_m2)) * 100

print("\n📊 MÉTRICAS DEL MODELO 2:")
print("=" * 50)
print(f"📈 R² Score:  {r2_m2:.3f}")
print(f"📉 RMSE:      ${rmse_m2:,.0f} ARS")
print(f"📊 MAE:       ${mae_m2:,.0f} ARS")
print(f"📐 MAPE:      {mape_m2:.2f}%")
print("=" * 50)

In [ ]:
# --- VISUALIZACIONES ---
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('MODELO 2: Predicción de Salario Ofrecido', fontsize=18, fontweight='bold', color='#4FC3F7')

# 1. Predicción vs Real
axes[0,0].scatter(y_test_m2, y_pred_m2, alpha=0.5, color='#4FC3F7')
axes[0,0].plot([y_test_m2.min(), y_test_m2.max()], 
               [y_test_m2.min(), y_test_m2.max()], 
               'r--', linewidth=2, label='Ideal')
axes[0,0].set_xlabel('Salario Real (ARS)')
axes[0,0].set_ylabel('Salario Predicho (ARS)')
axes[0,0].set_title(f'Predicción vs Real (R² = {r2_m2:.3f})', fontsize=14, color='#4FC3F7')
axes[0,0].legend()
axes[0,0].grid(alpha=0.3)

# 2. Residuos
residuos_m2 = y_test_m2 - y_pred_m2
axes[0,1].scatter(y_pred_m2, residuos_m2, alpha=0.5, color='#4FC3F7')
axes[0,1].axhline(y=0, color='r', linestyle='--', linewidth=2)
axes[0,1].set_xlabel('Predicción (ARS)')
axes[0,1].set_ylabel('Residuo (ARS)')
axes[0,1].set_title('Gráfico de Residuos', fontsize=14, color='#4FC3F7')
axes[0,1].grid(alpha=0.3)

# 3. Feature Importance
importances_m2 = pd.DataFrame({
    'feature': X_m2.columns,
    'importance': gbr_m2.feature_importances_
}).sort_values('importance', ascending=False).head(15)

axes[1,0].barh(range(len(importances_m2)), importances_m2['importance'], color='#4FC3F7')
axes[1,0].set_yticks(range(len(importances_m2)))
axes[1,0].set_yticklabels(importances_m2['feature'])
axes[1,0].set_xlabel('Importancia')
axes[1,0].set_title('Top 15 Features Más Importantes', fontsize=14, color='#4FC3F7')
axes[1,0].invert_yaxis()
axes[1,0].grid(axis='x', alpha=0.3)

# 4. Distribución de errores
axes[1,1].hist(residuos_m2, bins=30, color='#4FC3F7', edgecolor='black', alpha=0.7)
axes[1,1].axvline(x=0, color='r', linestyle='--', linewidth=2)
axes[1,1].set_xlabel('Error (ARS)')
axes[1,1].set_ylabel('Frecuencia')
axes[1,1].set_title('Distribución de Errores', fontsize=14, color='#4FC3F7')
axes[1,1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# --- GUARDAR RESULTADOS MODELO 2 ---
resultados_m2 = {
    'modelo': 'Predicción de Salario Ofrecido',
    'tipo': 'regresion',
    'algoritmo': 'Gradient Boosting Regressor',
    'metricas': {
        'r2_score': float(r2_m2),
        'rmse': float(rmse_m2),
        'mae': float(mae_m2),
        'mape': float(mape_m2)
    },
    'feature_importance': importances_m2.to_dict('records'),
    'estadisticas_salario': {
        'promedio': float(y_m2.mean()),
        'mediana': float(y_m2.median()),
        'min': float(y_m2.min()),
        'max': float(y_m2.max())
    },
    'dataset_size': {
        'total': len(df_m2),
        'train': len(X_train_m2),
        'test': len(X_test_m2)
    }
}

print("✅ Resultados guardados en variable 'resultados_m2'")

---
# 🌪️ MODELO 3: Impacto de Factores Externos en Ventas

**Objetivo**: Predecir tendencia de ventas según factores externos

**Variable target**: `ventas_tendencia` (-1=Peor, 0=Igual, 1=Mejor)

**Algoritmo**: Random Forest Classifier (multiclase)

In [ ]:
# --- PREPARAR DATOS ---
print("🌪️ MODELO 3: Impacto de Factores Externos\n")

features_m3 = [
    'afect_crimen_num',
    'afect_credito_num',
    'afect_precios_num',
    'afect_compe_num',
    'nivel_tecnologia',
    'antiguedad',
    'cantidad_trabajadores',
    'tiene_credito'
]

# Preparar dataset
df_m3 = data[features_m3 + ['ventas_tendencia', 'tipo_comercio']].copy()
df_m3 = df_m3.dropna()

# One-hot encoding
tipo_dummies_m3 = pd.get_dummies(df_m3['tipo_comercio'], prefix='tipo')
X_m3 = pd.concat([df_m3[features_m3], tipo_dummies_m3], axis=1)
y_m3 = df_m3['ventas_tendencia']

print(f"📊 Tamaño del dataset: {len(df_m3)} comercios")
print(f"\nDistribución de clases:")
print(y_m3.value_counts().sort_index())
print(f"\n📈 Mejor:  {(y_m3 == 1).sum()} ({(y_m3 == 1).mean()*100:.1f}%)")
print(f"➡️  Igual:  {(y_m3 == 0).sum()} ({(y_m3 == 0).mean()*100:.1f}%)")
print(f"📉 Peor:   {(y_m3 == -1).sum()} ({(y_m3 == -1).mean()*100:.1f}%)")

In [ ]:
# --- TRAIN/TEST SPLIT ---
X_train_m3, X_test_m3, y_train_m3, y_test_m3 = train_test_split(
    X_m3, y_m3, test_size=0.25, random_state=42, stratify=y_m3
)

print(f"🔀 Train set: {len(X_train_m3)} | Test set: {len(X_test_m3)}")

In [ ]:
# --- ENTRENAR MODELO ---
print("\n🤖 Entrenando Random Forest Classifier (Multiclase)...\n")

rf_m3 = RandomForestClassifier(
    n_estimators=200,
    max_depth=10,
    min_samples_split=10,
    random_state=42,
    n_jobs=-1
)

rf_m3.fit(X_train_m3, y_train_m3)

# Predicciones
y_pred_m3 = rf_m3.predict(X_test_m3)

print("✅ Modelo entrenado!")

In [ ]:
# --- MÉTRICAS ---
accuracy_m3 = accuracy_score(y_test_m3, y_pred_m3)
f1_weighted_m3 = f1_score(y_test_m3, y_pred_m3, average='weighted')

print("\n📊 MÉTRICAS DEL MODELO 3:")
print("=" * 50)
print(f"⚡ Accuracy:        {accuracy_m3:.3f}")
print(f"🔥 F1-Score (Weighted): {f1_weighted_m3:.3f}")
print("=" * 50)

# Classification Report
print("\n📋 Classification Report:")
print(classification_report(y_test_m3, y_pred_m3, 
                          target_names=['Peor', 'Igual', 'Mejor']))

In [ ]:
# --- VISUALIZACIONES ---
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('MODELO 3: Impacto de Factores Externos en Ventas', 
             fontsize=18, fontweight='bold', color='#4FC3F7')

# 1. Confusion Matrix
cm_m3 = confusion_matrix(y_test_m3, y_pred_m3)
sns.heatmap(cm_m3, annot=True, fmt='d', cmap='Blues', ax=axes[0,0],
            xticklabels=['Peor', 'Igual', 'Mejor'],
            yticklabels=['Peor', 'Igual', 'Mejor'])
axes[0,0].set_title('Matriz de Confusión', fontsize=14, color='#4FC3F7')
axes[0,0].set_ylabel('Real')
axes[0,0].set_xlabel('Predicho')

# 2. Feature Importance
importances_m3 = pd.DataFrame({
    'feature': X_m3.columns,
    'importance': rf_m3.feature_importances_
}).sort_values('importance', ascending=False).head(15)

axes[0,1].barh(range(len(importances_m3)), importances_m3['importance'], color='#4FC3F7')
axes[0,1].set_yticks(range(len(importances_m3)))
axes[0,1].set_yticklabels(importances_m3['feature'])
axes[0,1].set_xlabel('Importancia')
axes[0,1].set_title('Top 15 Features Más Importantes', fontsize=14, color='#4FC3F7')
axes[0,1].invert_yaxis()
axes[0,1].grid(axis='x', alpha=0.3)

# 3. Distribución de predicciones
pred_counts = pd.Series(y_pred_m3).value_counts().sort_index()
axes[1,0].bar(['Peor', 'Igual', 'Mejor'], pred_counts.values, color='#4FC3F7')
axes[1,0].set_ylabel('Cantidad')
axes[1,0].set_title('Distribución de Predicciones', fontsize=14, color='#4FC3F7')
axes[1,0].grid(axis='y', alpha=0.3)

# 4. Impacto promedio de afectaciones
afectaciones = ['afect_crimen_num', 'afect_credito_num', 'afect_precios_num', 'afect_compe_num']
impacto_promedio = df_m3.groupby('ventas_tendencia')[afectaciones].mean()

x_pos = np.arange(len(afectaciones))
width = 0.25

axes[1,1].bar(x_pos - width, impacto_promedio.loc[-1], width, label='Peor', color='#FF6B6B')
axes[1,1].bar(x_pos, impacto_promedio.loc[0], width, label='Igual', color='#FFB74D')
axes[1,1].bar(x_pos + width, impacto_promedio.loc[1], width, label='Mejor', color='#00E676')

axes[1,1].set_xticks(x_pos)
axes[1,1].set_xticklabels(['Crimen', 'Crédito', 'Precios', 'Competencia'], rotation=45)
axes[1,1].set_ylabel('Nivel de Afectación Promedio (0-3)')
axes[1,1].set_title('Afectaciones por Tendencia de Ventas', fontsize=14, color='#4FC3F7')
axes[1,1].legend()
axes[1,1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# --- GUARDAR RESULTADOS MODELO 3 ---
resultados_m3 = {
    'modelo': 'Impacto de Factores Externos en Ventas',
    'tipo': 'clasificacion_multiclase',
    'algoritmo': 'Random Forest Classifier',
    'metricas': {
        'accuracy': float(accuracy_m3),
        'f1_weighted': float(f1_weighted_m3)
    },
    'feature_importance': importances_m3.to_dict('records'),
    'confusion_matrix': cm_m3.tolist(),
    'distribucion_clases': y_m3.value_counts().sort_index().to_dict(),
    'dataset_size': {
        'total': len(df_m3),
        'train': len(X_train_m3),
        'test': len(X_test_m3)
    }
}

print("✅ Resultados guardados en variable 'resultados_m3'")

---
# 🏪 MODELO 4: Score de Viabilidad Comercial

**Objetivo**: Agrupar comercios según su salud/viabilidad usando clustering

**Método**: K-Means Clustering + Score compuesto

**Dimensiones del score**: Expectativas, Crédito, Inventario, Antigüedad, Trabajadores

In [ ]:
# --- PREPARAR DATOS ---
print("🏪 MODELO 4: Score de Viabilidad\n")

features_m4 = [
    'expectativas_ventas_num',
    'tiene_credito',
    'nivel_tecnologia',
    'antiguedad',
    'cantidad_trabajadores',
    'horas_operacion',
    'local_propio'
]

# Preparar dataset
df_m4 = data[features_m4].copy()
df_m4 = df_m4.dropna()

print(f"📊 Tamaño del dataset: {len(df_m4)} comercios")

In [ ]:
# --- NORMALIZAR ---
scaler_m4 = StandardScaler()
X_m4_scaled = scaler_m4.fit_transform(df_m4)

print("✅ Datos normalizados")

In [ ]:
# --- CLUSTERING K-MEANS ---
print("\n🤖 Aplicando K-Means (k=3)...\n")

kmeans_m4 = KMeans(n_clusters=3, random_state=42, n_init=10)
clusters_m4 = kmeans_m4.fit_predict(X_m4_scaled)

df_m4['cluster'] = clusters_m4

print("✅ Clustering completado!")
print(f"\nDistribución de clusters:")
print(pd.Series(clusters_m4).value_counts().sort_index())

In [ ]:
# --- CALCULAR SCORE DE VIABILIDAD ---

def calcular_score(row):
    score = 0
    
    # Expectativas positivas (0-30 puntos)
    if row['expectativas_ventas_num'] == 1:
        score += 30
    elif row['expectativas_ventas_num'] == 0:
        score += 15
    
    # Acceso a crédito (0-20 puntos)
    if row['tiene_credito'] == 1:
        score += 20
    
    # Tecnología (0-20 puntos)
    score += (row['nivel_tecnologia'] / 3) * 20
    
    # Antigüedad (0-15 puntos) - más antigüedad = más estable
    antiguedad_norm = min(row['antiguedad'] / 20, 1)  # Cap a 20 años
    score += antiguedad_norm * 15
    
    # Trabajadores (0-10 puntos)
    trabajadores_norm = min(row['cantidad_trabajadores'] / 10, 1)
    score += trabajadores_norm * 10
    
    # Local propio (0-5 puntos)
    if row['local_propio'] == 1:
        score += 5
    
    return round(score, 1)

df_m4['score_viabilidad'] = df_m4.apply(calcular_score, axis=1)

print("📊 Score de Viabilidad calculado")
print(f"\nEstadísticas del score:")
print(df_m4['score_viabilidad'].describe())

In [ ]:
# --- ETIQUETAR CLUSTERS ---
# Ordenar clusters por score promedio
cluster_scores = df_m4.groupby('cluster')['score_viabilidad'].mean().sort_values(ascending=False)
cluster_labels = {cluster_scores.index[0]: 'Alto', 
                 cluster_scores.index[1]: 'Medio', 
                 cluster_scores.index[2]: 'Bajo'}

df_m4['nivel_viabilidad'] = df_m4['cluster'].map(cluster_labels)

print("\n🏷️ Clusters etiquetados:")
print(df_m4['nivel_viabilidad'].value_counts())

print("\n📊 Score promedio por nivel:")
print(df_m4.groupby('nivel_viabilidad')['score_viabilidad'].mean().sort_values(ascending=False))

In [ ]:
# --- VISUALIZACIONES ---
fig = plt.figure(figsize=(18, 12))
gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)
fig.suptitle('MODELO 4: Score de Viabilidad Comercial', 
             fontsize=18, fontweight='bold', color='#4FC3F7')

# 1. PCA 2D de clusters
ax1 = fig.add_subplot(gs[0, :])
pca_m4 = PCA(n_components=2)
X_pca_m4 = pca_m4.fit_transform(X_m4_scaled)
df_pca = pd.DataFrame(X_pca_m4, columns=['PC1', 'PC2'])
df_pca['cluster'] = df_m4['nivel_viabilidad'].values

colors = {'Alto': '#00E676', 'Medio': '#4FC3F7', 'Bajo': '#FF6B6B'}
for nivel in ['Alto', 'Medio', 'Bajo']:
    mask = df_pca['cluster'] == nivel
    ax1.scatter(df_pca[mask]['PC1'], df_pca[mask]['PC2'], 
               label=nivel, alpha=0.6, s=100, color=colors[nivel])

ax1.set_xlabel(f'PC1 ({pca_m4.explained_variance_ratio_[0]*100:.1f}% var)')
ax1.set_ylabel(f'PC2 ({pca_m4.explained_variance_ratio_[1]*100:.1f}% var)')
ax1.set_title('Visualización PCA de Clusters', fontsize=14, color='#4FC3F7')
ax1.legend()
ax1.grid(alpha=0.3)

# 2. Distribución de scores por cluster
ax2 = fig.add_subplot(gs[1, 0])
for nivel in ['Alto', 'Medio', 'Bajo']:
    data_nivel = df_m4[df_m4['nivel_viabilidad'] == nivel]['score_viabilidad']
    ax2.hist(data_nivel, bins=20, alpha=0.6, label=nivel, color=colors[nivel])
ax2.set_xlabel('Score de Viabilidad')
ax2.set_ylabel('Frecuencia')
ax2.set_title('Distribución de Scores', fontsize=14, color='#4FC3F7')
ax2.legend()
ax2.grid(alpha=0.3)

# 3. Boxplot de scores
ax3 = fig.add_subplot(gs[1, 1])
df_m4.boxplot(column='score_viabilidad', by='nivel_viabilidad', ax=ax3)
ax3.set_xlabel('Nivel de Viabilidad')
ax3.set_ylabel('Score')
ax3.set_title('Comparación de Scores', fontsize=14, color='#4FC3F7')
plt.sca(ax3)
plt.xticks([1, 2, 3], ['Alto', 'Medio', 'Bajo'])

# 4. Conteo por cluster
ax4 = fig.add_subplot(gs[1, 2])
counts = df_m4['nivel_viabilidad'].value_counts()
ax4.bar(['Alto', 'Medio', 'Bajo'], 
        [counts.get('Alto', 0), counts.get('Medio', 0), counts.get('Bajo', 0)],
        color=[colors['Alto'], colors['Medio'], colors['Bajo']])
ax4.set_ylabel('Cantidad de Comercios')
ax4.set_title('Distribución por Nivel', fontsize=14, color='#4FC3F7')
ax4.grid(axis='y', alpha=0.3)

# 5. Radar chart de características por cluster
ax5 = fig.add_subplot(gs[2, :], projection='polar')

# Promedios por cluster (normalizados 0-1)
features_radar = ['expectativas_ventas_num', 'tiene_credito', 'nivel_tecnologia', 
                 'antiguedad', 'cantidad_trabajadores']

for nivel in ['Alto', 'Medio', 'Bajo']:
    valores = df_m4[df_m4['nivel_viabilidad'] == nivel][features_radar].mean()
    # Normalizar entre 0 y 1
    valores_norm = (valores - valores.min()) / (valores.max() - valores.min() + 0.001)
    
    angles = np.linspace(0, 2 * np.pi, len(features_radar), endpoint=False).tolist()
    valores_norm = valores_norm.tolist()
    valores_norm += valores_norm[:1]
    angles += angles[:1]
    
    ax5.plot(angles, valores_norm, 'o-', linewidth=2, label=nivel, color=colors[nivel])
    ax5.fill(angles, valores_norm, alpha=0.15, color=colors[nivel])

ax5.set_xticks(angles[:-1])
ax5.set_xticklabels(['Expectativas', 'Crédito', 'Tecnología', 'Antigüedad', 'Trabajadores'])
ax5.set_ylim(0, 1)
ax5.set_title('Perfil de Características por Nivel', fontsize=14, color='#4FC3F7', pad=20)
ax5.legend(loc='upper right', bbox_to_anchor=(1.3, 1.0))
ax5.grid(True)

plt.show()

In [ ]:
# --- GUARDAR RESULTADOS MODELO 4 ---
resultados_m4 = {
    'modelo': 'Score de Viabilidad Comercial',
    'tipo': 'clustering',
    'algoritmo': 'K-Means (k=3)',
    'distribucion_clusters': df_m4['nivel_viabilidad'].value_counts().to_dict(),
    'score_promedio_por_nivel': df_m4.groupby('nivel_viabilidad')['score_viabilidad'].mean().to_dict(),
    'estadisticas_score': {
        'min': float(df_m4['score_viabilidad'].min()),
        'max': float(df_m4['score_viabilidad'].max()),
        'mean': float(df_m4['score_viabilidad'].mean()),
        'median': float(df_m4['score_viabilidad'].median())
    },
    'varianza_explicada_pca': {
        'PC1': float(pca_m4.explained_variance_ratio_[0]),
        'PC2': float(pca_m4.explained_variance_ratio_[1])
    },
    'dataset_size': len(df_m4)
}

print("✅ Resultados guardados en variable 'resultados_m4'")

---
# 💾 EXPORTAR RESULTADOS PARA LA WEB

In [ ]:
# --- CONSOLIDAR TODOS LOS RESULTADOS ---
resultados_completos = {
    'metadata': {
        'fecha_analisis': pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S'),
        'dataset_original': {
            'filas': len(df),
            'columnas': len(df.columns)
        },
        'equipo': 'GreenThunder',
        'institucion': 'MIT LIFT Lab × UBA'
    },
    'modelos': {
        'modelo_1_crecimiento': resultados_m1,
        'modelo_2_salario': resultados_m2,
        'modelo_3_factores_externos': resultados_m3,
        'modelo_4_viabilidad': resultados_m4
    }
}

# Guardar JSON
with open('ml_results.json', 'w', encoding='utf-8') as f:
    json.dump(resultados_completos, f, indent=2, ensure_ascii=False)

print("\n✅ RESULTADOS EXPORTADOS")
print("=" * 50)
print("📁 Archivo: ml_results.json")
print(f"📊 Tamaño: {len(json.dumps(resultados_completos))/1024:.1f} KB")
print("\n🎯 Listo para integrar en la web!")

---
# 📊 RESUMEN FINAL

In [ ]:
print("\n" + "="*70)
print("🎯 RESUMEN DE MODELOS - MACHINE LEARNING COMERCIOS".center(70))
print("="*70 + "\n")

print("📈 MODELO 1: Predicción de Crecimiento Comercial")
print(f"   • Accuracy:  {accuracy_m1:.3f}")
print(f"   • AUC-ROC:   {auc_m1:.3f}")
print(f"   • Dataset:   {len(df_m1)} comercios\n")

print("💰 MODELO 2: Predicción de Salario Ofrecido")
print(f"   • R² Score:  {r2_m2:.3f}")
print(f"   • RMSE:      ${rmse_m2:,.0f} ARS")
print(f"   • MAPE:      {mape_m2:.2f}%")
print(f"   • Dataset:   {len(df_m2)} comercios\n")

print("🌪️ MODELO 3: Impacto de Factores Externos")
print(f"   • Accuracy:  {accuracy_m3:.3f}")
print(f"   • F1-Score:  {f1_weighted_m3:.3f}")
print(f"   • Dataset:   {len(df_m3)} comercios\n")

print("🏪 MODELO 4: Score de Viabilidad")
print(f"   • Clusters:  3 niveles (Alto/Medio/Bajo)")
print(f"   • Score promedio: {df_m4['score_viabilidad'].mean():.1f}/100")
print(f"   • Dataset:   {len(df_m4)} comercios\n")

print("="*70)
print("✅ ANÁLISIS COMPLETADO".center(70))
print("="*70)